In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Project WeRateDogs Twitter Data: Wrangle and Analyze Data

# importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import json
#import tweepy
import os
import re
import requests

# Gathering data

In [ ]:
#I will gather data from a variety of sources and in a variety of formats:

#from :

#1- Download this file manually: 
#This file (twitter_archive_enhanced.csv)
#The WeRateDogs Twitter archive,

#2- Downloaded programmatically using the requests library and the following 
#URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv  
#This file (image_predictions.tsv)
#The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.)
# is present in each tweet according to a neural network is hosted on Udacity's servers

#3- Query the Twitter API :
#for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data
#in a file called tweet_json.txt file.
#This file (tweet_json.txt)
#Each tweet's JSON data should be written to its own line.
#Then 
#Reading this file line by line into a pandas DataFrame with  tweet ID, retweet count, and favorite count. 

 # 1- Download this file manually: This file (twitter_archive_enhanced.csv)

In [ ]:
df_twt = pd.read_csv('../input/weratedogs/twitter-archive-enhanced.csv')

In [ ]:
df_twt.sample(3)

# 2- Downloaded manually: This file (image_predictions.tsv)

In [ ]:
df_img = pd.read_csv('../input/image-predictions/image_predictions.tsv', sep = '\t')

In [ ]:
df_img.sample(1)

# 3- Additional Data via the Twitter API: This file ( tweet_json.txt)¶
# Accessing Project Data Without a Twitter Account
# I can't set up a Twitter developer account so i instead follow the directions below by udacity to access the data necessary for the project.

In [ ]:
# Reading the json file
id_twt = []
num_fav = []
num_ret = []
with open('../input/tweet-json/tweet_json.txt', mode = 'r') as f:
     for line in f.readlines():
            tweet_data = json.loads(line)
            id_twt.append(tweet_data['id'])
            num_fav.append(tweet_data['favorite_count'])
            num_ret.append(tweet_data['retweet_count'])
            
df_json = pd.DataFrame({'tweet_id':id_twt, 'favorite_count':num_fav, 'retweet_count':num_ret})

In [ ]:
df_json.sample(1)

In [ ]:
df_json.info()

In [ ]:
df_twt.tail(1)

In [ ]:
df_img.tail(1)

In [ ]:
df_json.tail(1)

# **#NOW I have 3 dataframes
# #df_twt = 0 - 2355 = 2356 rows
# #df_img = 0 - 2074 = 2075 rows
# #df_json = 0 - 2353 = 2354 rows

# #for sure**

In [ ]:
df_twt.shape

In [ ]:
df_img.shape

In [ ]:
df_json.shape

# Assessing data

In [ ]:
#Assess its quality and tidiness
#visually and programmatically
#I will document at least eight (8) quality issues and two (2) tidiness issues
# such as requested.
#I created a Python function that includes the required functions for assess
#from what I learned in this course for easy and speed in performance

In [ ]:
def assess_data(topic, source_df, df):
    print('Use pandas to explore and assess {} datasets  {}  in terms of quality \
    and tidness \n by display data details:'.format(topic, source_df))
    print('----------------------------------------------------------------')
    print('Number of columns and rows :')
    print(df.shape)
    print('---------------------------')
    print('The first 5 lines of the data frame :')
    print(df.head())
    print('----------------------------------------------------------------')
    print('The last 5 lines of the data frame :')
    print(df.tail())
    print('----------------------------------------------------------------')
    print('The sample 7 lines of the data frame :')
    print(df.sample(7))
    print('----------------------------------------------------------------')
    print('Duplicate rows in the {} {} dataset'.format(topic, source_df))
    print(df.duplicated().sum())
    print('----------------------------------------------------------------')
    print('Datatype of column in the {} {} dataset'.format(topic, source_df))
    print(df.info())
    print('----------------------------------------------------------------')
    print('Features with missing values in the {} {} dataset'.format(topic, source_df))
    print(df.isnull().sum())
    print('----------------------------------------------------------------')
    print('Number of unique values for quality in the {} {} dataset'.format(topic, source_df))
    print(df.nunique())
    print('----------------------------------------------------------------')
    print('Number of rows with missing values in the {} {} dataset'.format(topic, source_df))
    print(df.isnull().any(axis=1).sum())
    print('----------------------------------------------------------------')

In [ ]:
#assess
## run : def
if __name__ == '__main__':
    t = 'We_Rate_Dogs'
    s1 = 'from Twitter archive file'
    s2 = 'from Tweet Image Prediction'
    s3 = 'from Twitter API and JSON'

    print('--------------------------------------------------------------')
    print('1 - Inspect to Assess dataframe of Twitter archive file = df_twt :')
    assess_data(t, s1, df_twt)
    print('--------------------------------------------------------------')
    print('2- Inspect to Assess dataframe of Tweet Image Prediction = df_img :')
    assess_data(t, s2, df_img)
    print('--------------------------------------------------------------')
    print('3- Inspect to Assess dataframe of Twitter API and JSON = df_json :')
    assess_data(t, s3, df_json)
    print('--------------------------------------------------------------') 

# Document at least eight (8) quality issues and two (2) tidiness issues

In [ ]:
#Tidiness issue 2 : Messy data, also known as untidy data. Untidy data has structural issues.
#1- all data frames must be integrated into one frame with a common factor, tweet_id, to be easy to clean
#2- each dog must have one stage, so we will create one column for the stage and ignore the null values

##Quality Issues 8 : Dirty data, also known as low quality data. Low quality data has content issues.
#1- I will drop the columns for the 4 stages as they become unnecessary or duplicate
#2-  I will drop the columns :  'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp'
    #I will keep only those rows in  archieve table that are original tweets and not retweets
#3-  I will convert tweet_id to str
#4- I will convert timestamp to datetime to be suitable
#5- I will remove columns that are not needed for analysis
#6- I will drop duplicate jpg_url  2075 - 2009 = 66
  #To remove duplicates on specific column(s), use subset, to remove duplicates and keep last occurences, use keep
#7- I will put a space in place of the underscore for both p1,p2 and p3
#8- I will convert rating_numerator, rating_denominator to float to be suitable

# Cleaning data

In [ ]:
#clean
#I will clean each of the issues documented while assessing.

#The result will be a high quality and tidy master pandas DataFrame.

# Copies of original dataframes to clean. 

twt_clean = df_twt.copy()
json_clean = df_json.copy()
img_clean = df_img.copy()

In [ ]:
#Tidiness issue 2

In [ ]:
# Tidiness(1) : Merge 3 dataframes in one dataframe :

#Define:
#First, all data frames must be integrated into one frame with a common factor, tweet_id, to be easy to clean

#Code:
twt_clean = pd.merge(twt_clean, json_clean, on = ['tweet_id'], how = 'inner')
twt_clean = pd.merge(twt_clean, img_clean, on ='tweet_id', how = 'inner')

#Test:
twt_clean.info()

In [ ]:
# Tidiness(2) : Merge ['doggo', 'floofer', 'pupper', 'puppo'] in one Column :

#Define:
#Second, each dog must have one stage, so we will create one column for the stage and ignore the null values 


#Code:
twt_clean['stage'] = twt_clean[['doggo', 'floofer', 'pupper', 'puppo']].max(axis=1)

#Test:
twt_clean.stage.value_counts().sum()

In [ ]:
twt_clean.info()

In [ ]:
#become 15 object

In [ ]:
#Quality Issues 8 :

In [ ]:
# Quality(1) 

#Define:
# I will drop the columns for the 4 stages as they become unnecessary or duplicate 

#Code:
twt_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

#Test:
twt_clean.info()

In [ ]:
# Quality(2) 

#Define:
# I will drop the columns :  'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp'
#I will keep only those rows in  archieve table that are original tweets and not retweets

#Code:
twt_clean = twt_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

#Test:
twt_clean.info()

In [ ]:
# Quality(3) 

#Define:
# I will convert tweet_id to str

#Code:
twt_clean['tweet_id'] = twt_clean['tweet_id'].astype(str)

#Test:
twt_clean.info()

In [ ]:
# Quality(4) 

#Define:
# I will convert timestamp to datetime to be suitable

#Code:
twt_clean.timestamp = pd.to_datetime(twt_clean.timestamp)

#Test:
twt_clean.info()

In [ ]:
# Quality(5) 

#Define:
# I will remove columns that are not needed for analysis

#Code:
twt_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace= True)

#Test:
twt_clean.columns

In [ ]:
# Quality(6) 

#Define:
# I will drop duplicate jpg_url  2075 - 2009 = 66
#To remove duplicates on specific column(s), use subset, to remove duplicates and keep last occurences, use keep

#Code:
twt_clean = twt_clean.drop_duplicates(subset=['jpg_url'], keep='last')

#Test:
sum(twt_clean['jpg_url'].duplicated())

In [ ]:
# Quality(7) 

#Define:
#I will put a space in place of the underscore for both p1,p2 and p3

#Code:
twt_clean['p1'] = twt_clean['p1'].str.replace('_', ' ')
twt_clean['p2'] = twt_clean['p2'].str.replace('_', ' ')
twt_clean['p3'] = twt_clean['p3'].str.replace('_', ' ')

#Test:
twt_clean.head()

In [ ]:
# Quality(8) 

#Define:
# I will convert rating_numerator, rating_denominator to float to be suitable

#Code:
twt_clean['rating_numerator'] = twt_clean['rating_numerator'].astype(float)
twt_clean['rating_denominator'] = twt_clean['rating_denominator'].astype(float)

#Test:
twt_clean.info()

# Storing data

In [ ]:
twt_master = twt_clean.copy()
# I will save twt_master dataframe to a CSV file
twt_master.to_csv('twitter_archive_master.csv')

# Analyzing and Visualizing

In [ ]:
#statistics 
twt_master.describe()

In [ ]:
twt_master.p1.value_counts()

In [ ]:
twt_master.p2.value_counts()

In [ ]:
twt_master.p3.value_counts()

In [ ]:
#This indicates that the most prediction is to golden retriever and Labrador retriever

In [ ]:
twt_master['rating_denominator'].value_counts()

In [ ]:
twt_master['rating_denominator'].value_counts().plot(kind='bar');

In [ ]:
#This indicates that the most rating_denominator is 10 to fact dog

In [ ]:
#to display favorite_count vs stage
plt.scatter(twt_master['favorite_count'], twt_master['stage'])
plt.xlabel('favorite_count')
plt.ylabel('stage')
plt.title('stages and favorites by Scatter plot')

In [ ]:
#This indicates that the most favorites stage is pupper

In [ ]:
#to display favorite_count vs retweet_count
plt.scatter(twt_master['favorite_count'], twt_master['retweet_count'])
plt.xlabel('favorite_count')
plt.ylabel('retweet_count')
plt.title('Retweets and favorites by Scatter plot')

In [ ]:
#to display p_conf is how confident the algorithm is in its prediction
#for p1,p2 and p3

In [ ]:
plt.hist(x = twt_master.p1_conf, bins = 80)
plt.show()

In [ ]:
plt.hist(x = twt_master.p2_conf, bins = 80)
plt.show()

In [ ]:
plt.hist(x = twt_master.p3_conf, bins = 80)
plt.show()

In [ ]:
# plot stage of dog
twt_master['stage'].hist();

In [ ]:
#This indicates that the most fact stages is pupper 

In [ ]:
twt_master['stage'].describe()

# #visualizations  with alot of details and Transparency and clarity 
# #by using seaborn

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv('twitter_archive_master.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Report (1)
# 2008 rows
# 23 columns
# there values by null but it no display because none

In [ ]:
df['rating_denominator'].value_counts().plot(kind='bar');

In [ ]:
#This indicates that the most rating_denominator is 10 to fact dog

In [ ]:
#to display favorite_count vs stage
plt.scatter(df['favorite_count'], df['stage'])
plt.xlabel('favorite_count')
plt.ylabel('stage')
plt.title('stages and favorites by Scatter plot')

In [ ]:
#This indicates that the most favorites stage is pupper

In [ ]:
#to display favorite_count vs retweet_count
plt.scatter(df['favorite_count'], df['retweet_count'])
plt.xlabel('favorite_count')
plt.ylabel('retweet_count')
plt.title('Retweets and favorites by Scatter plot')

In [ ]:
#to display p_conf is how confident the algorithm is in its prediction
#for p1,p2 and p3

In [ ]:
plt.hist(x = df.p1_conf, bins = 80)
plt.show()

In [ ]:
plt.hist(x = df.p2_conf, bins = 80)
plt.show()

In [ ]:
plt.hist(x = df.p3_conf, bins = 80)
plt.show()

In [ ]:
# plot stage of dog
df['stage'].hist();

In [ ]:
#This indicates that the most fact stages is pupper 

In [ ]:
df['img_num'].value_counts().plot(kind='bar');

In [ ]:
df['img_num'].value_counts().plot(kind='pie');

In [ ]:
df['stage'].value_counts().plot(kind='pie');

# #This indicates also that the most fact stages is pupper 

# # job great with udacity  by Hedra Atia